In [1]:
# hide
%load_ext autoreload
%autoreload 2

# NumerBlox

> Solid Numerai pipelines

`numerblox` offers Numerai specific functionality, so you have to worry less about software/data engineering and can focus more on building great models!

Most of the components in this library are designed for solid weekly inference pipelines, but tools like `NumerFrame` can also greatly simplify the training process.

Documentation: [crowdcent.github.io/numerblox](https://crowdcent.github.io/numerblox/)

## 1. Install

`pip install numerblox`

## 2. How to use

### 2.1. Contents

### 2.1.1. Core functionality

`numerblox` features the following functionality:

1. Downloading data (`NumeraiClassicDownloader` and `KaggleDownloader`)
2. A custom data structure extending Pandas DataFrame (`NumerFrame`)
3. A suite of preprocessors for Numerai Classic and Signals (feature selection, engineering and manipulation)
4. Model objects for easy inference.
5. A suite of postprocessors for Numerai Classic and Signals (standardization, ensembling, neutralization and penalization)
6. Pipelines handling processing and prediction (`ModelPipeline` and `ModelPipelineCollection`)
7. Evaluation (`NumeraiClassicEvaluator` and `NumeraiSignalsEvaluator`)
8. Authentication (`Key` and `load_key_from_json`)
9. Submitting (`NumeraiClassicSubmitter` and `NumeraiSignalsSubmitter`)
10. Automated staking (`NumeraiClassicStaker` and `NumeraiSignalsStaker`)

### 2.1.2. Educational notebooks

Example notebooks can be found in the `edu_nbs` directory.

`edu_nbs` currently contains the following examples:
- `numerframe_tutorial.ipynb`: A deep dive into what `NumerFrame` has to offer.
- `pipeline_construction.ipynb`: How to use `numerblox` tools for efficient Numerai inference.
- `submitting.ipynb`: How to use Submitters for safe and easy Numerai submissions.
- `google_cloud_storage.ipynb`: How to use Downloaders and Submitters to interact with Google Cloud Storage (GCS).
- `load_model_from_wandb.ipynb`: For [Weights & Biases](https://wandb.ai/) users. Easily pull a model from W&B for inference.

Development notebooks are in the `nbs` directory. These notebooks are also used to generate the documentation.

**Full documentation:** [crowdcent.github.io/numerblox](https://crowdcent.github.io/numerblox/)

### 2.2. Examples

Below we will illustrate a common use case for inference pipelines. To learn more in-depth about the features of this library, check out notebooks in `edu_nbs`.

#### 2.2.1. Numerai Classic

In [1]:
#other
#hide_output

# --- 0. Numerblox dependencies ---
from numerblox.download import NumeraiClassicDownloader
from numerblox.numerframe import create_numerframe
from numerblox.postprocessing import FeatureNeutralizer
from numerblox.model import SingleModel
from numerblox.model_pipeline import ModelPipeline
from numerblox.key import load_key_from_json
from numerblox.submission import NumeraiClassicSubmitter

# --- 1. Download version 2 data ---
downloader = NumeraiClassicDownloader("data")
downloader.download_inference_data("current_round")

# --- 2. Initialize NumerFrame ---
metadata = {"version": 2,
            "joblib_model_name": "test",
            "joblib_model_path": "test_assets/joblib_v2_example_model.joblib",
            "numerai_model_name": "test_model1",
            "key_path": "test_assets/test_credentials.json"}
dataf = create_numerframe(file_path="data/current_round/numerai_tournament_data.parquet",
                          metadata=metadata)

# --- 3. Define and run pipeline ---
models = [SingleModel(dataf.meta.joblib_model_path,
                      model_name=dataf.meta.joblib_model_name)]
# No preprocessing and 0.5 feature neutralization
postprocessors = [FeatureNeutralizer(pred_name=f"prediction_{dataf.meta.joblib_model_name}",
                                     proportion=0.5)]
pipeline = ModelPipeline(preprocessors=[],
                         models=models,
                         postprocessors=postprocessors)
dataf = pipeline(dataf)

# --- 4. Submit ---
# Load credentials from .json (random credentials in this example)
key = load_key_from_json(dataf.meta.key_path)
submitter = NumeraiClassicSubmitter(directory_path="sub_current_round", key=key)
# full_submission checks contents, saves as csv and submits.
submitter.full_submission(dataf=dataf,
                          cols=f"prediction_{dataf.meta.joblib_model_name}_neutralized_0.5",
                          model_name=dataf.meta.numerai_model_name,
                          version=dataf.meta.version)

# --- 5. Clean up environment (optional) ---
downloader.remove_base_directory()
submitter.remove_base_directory()

In [2]:
# hide_input
from rich.console import Console
from rich.tree import Tree

console = Console(record=True, width=100)

tree = Tree(":computer: Directory structure before starting", guide_style="bold bright_black")
model_tree = tree.add(":file_folder: test_assets")
model_tree.add(":page_facing_up: joblib_v2_example_model.joblib")
model_tree.add(":page_facing_up: test_credentials.json")

console.print(tree)

tree2 = Tree(":computer: Directory structure after submitting", guide_style="bold bright_black")
data_tree = tree2.add(":file_folder: data")
current_tree = data_tree.add(":file_folder: current_round")
current_tree.add(":page_facing_up: numerai_tournament_data.parquet")
sub_tree = tree2.add(":file_folder: sub_current_round")
sub_tree.add(":page_facing_up: test_model1.csv")
model_tree = tree.add(":file_folder: test_assets")
model_tree.add(":page_facing_up: joblib_v2_example_model.joblib")
model_tree.add(":page_facing_up: test_credentials.json")

console.print(tree2)

💻 Directory structure before starting                                                              
┗━━ 📁 test_assets                                                                                  
    ┣━━ 📄 joblib_v2_example_model.joblib                                                           
    ┗━━ 📄 test_credentials.json

💻 Directory structure after submitting                                                             
┣━━ 📁 data                                                                                         
┃   ┗━━ 📁 current_round                                                                            
┃       ┗━━ 📄 numerai_tournament_data.parquet                                                      
┗━━ 📁 sub_current_round                                                                            
    ┗━━ 📄 test_model1.csv

### 2.2.2. Numerai Signals

In [ ]:
# other
# hide_output

# --- 0. Numerblox dependencies ---
from numerblox.download import KaggleDownloader
from numerblox.numerframe import create_numerframe
from numerblox.preprocessing import KatsuFeatureGenerator
from numerblox.submission import NumeraiSignalsSubmitter

# --- 1. Download Katsu1110 yfinance dataset from Kaggle ---
kd = KaggleDownloader("data")
kd.download_inference_data("code1110/yfinance-stock-price-data-for-numerai-signals")

# --- 2. Initialize NumerFrame with metadata ---
metadata = {"numerai_model_name": "test_model1",
            "key_path": "test_assets/test_credentials.json"}
dataf = create_numerframe("data/full_data.parquet", metadata=metadata)

# --- 3. Define and run pipeline ---
models = [SingleModel("models/signals_model.cbm", model_name="cb")]
# Simple and fast feature generator based on Katsu Signals starter notebook
# https://www.kaggle.com/code1110/numeraisignals-starter-for-beginners
pipeline = ModelPipeline(preprocessors=[KatsuFeatureGenerator(windows=[20, 40, 60])],
                         models=models,
                         postprocessors=[])
dataf = pipeline(dataf)

# --- 4. Submit ---
# Load credentials from .json (random credentials in this example)
key = load_key_from_json(dataf.meta.key_path)
submitter = NumeraiSignalsSubmitter(directory_path="sub_current_round", key=key)
# full_submission checks contents, saves as csv and submits.
# cols selection must at least contain 1 ticker column and a signal column.
dataf['signal'] = dataf['prediction_cb']
submitter.full_submission(dataf=dataf,
                          cols=['bloomberg_ticker', 'signal'],
                          model_name=dataf.meta.numerai_model_name)

# --- 5. Clean up environment (optional) ---
kd.remove_base_directory()
submitter.remove_base_directory()

In [3]:
# hide_input
from rich.console import Console
from rich.tree import Tree

console = Console(record=True, width=100)

tree = Tree(":computer: Directory structure before starting", guide_style="bold bright_black")
model_tree = tree.add(":file_folder: test_assets")
models_tree = tree.add(":file_folder: models")
models_tree.add(":page_facing_up: signals_model.cbm")
model_tree.add(":page_facing_up: test_credentials.json")

console.print(tree)

tree2 = Tree(":computer: Directory structure after submitting", guide_style="bold bright_black")
data_tree = tree2.add(":file_folder: data")
data_tree.add(":page_facing_up: full_data.parquet")
sub_tree = tree2.add(":file_folder: sub_current_round")
sub_tree.add(":page_facing_up: submission.csv")
model_tree = tree.add(":file_folder: test_assets")
model_tree.add(":page_facing_up: test_credentials.json")
models_tree = tree.add(":file_folder: models")
models_tree.add(":page_facing_up: signals_model.cbm")

console.print(tree2)

💻 Directory structure before starting                                                              
┣━━ 📁 test_assets                                                                                  
┃   ┗━━ 📄 test_credentials.json                                                                    
┗━━ 📁 models                                                                                       
    ┗━━ 📄 signals_model.cbm

💻 Directory structure after submitting                                                             
┣━━ 📁 data                                                                                         
┃   ┗━━ 📄 full_data.parquet                                                                        
┗━━ 📁 sub_current_round                                                                            
    ┗━━ 📄 submission.csv

## 3. Contributing

### 3.1. Overview

Below are a few guidelines for development of `numerblox`. Also be sure to read `CONTRIBUTING.md` for more detailed instruction on contributing.

Thanks a lot for wanting to help us out with this project! We are using a project setup called [nbdev](https://nbdev.fast.ai/) to easily develop code, documentation and tests within Jupyter notebooks. If you are only using the library you don't have to worry about this. Just pip install and you are good to go!

If you are thinking of contributing and are not familiar with nbdev, it may take some time to learn nbdev development. We are happy to help out and point you to documentation or videos to learn more.

If you are interested in the full scope of what nbdev has to offer, check out this tutorial with Jeremy Howard:
 [https://youtu.be/Hrs7iEYmRmg](https://youtu.be/Hrs7iEYmRmg).

Why are we using nbdev? To learn more about the rationale behind nbdev:
[https://youtu.be/9Q6sLbz37gk](https://youtu.be/9Q6sLbz37gk)

nbdev live coding example with Hamel Husain:
[https://youtu.be/ZJTop5uqC2U](https://youtu.be/ZJTop5uqC2U)



### 3.2. Bugs / Issues / Enhancements.

Even though most of the components in this library are tested, the project is still in an early stage of development. If you discover bugs, other issues or ideas for enhancements, do not hesitate to make a Github issue. Describe in the issue what code was run on what machine and background on the issue. Add stacktraces and screenshots if this is relevant for solving the issue. Also, please add appropriate labels for the Github issue.

### 3.3. Contributing Code

There are a few small things you should do before contributing code to this project. After you clone the repository, please run `nbdev_install_git_hooks` in your terminal. This sets up git hooks, which cleans up the notebooks to remove the extraneous stuff stored in the notebooks (e.g. which cells you ran). This avoids unnecessary merge conflicts.

When adding a new feature, only change code in the `nbs/` directory. Then, before pushing code, be sure to run `nbdev_build_lib` and `nbdev_build_docs` so the notebooks and source code are synced. `nbdev` automatically handles the parsing of notebook code into source code and automatically generates documentation from these notebooks.



### 3.4. Branch structure


Every new feature should be implemented in a branch that branches from `dev` and has the naming convention `feature/{FEATURE_DESCRIPTION}`. Explicit bugfixes should be named `bugfix/{FIX_DESCRIPTION}`. An example structure is given below.

In [3]:
# hide_input
console = Console(record=True, width=100)

tree = Tree("Branch structure", guide_style="bold bright_black")

main_tree = tree.add("📦 main (release)", guide_style="bright_black")
dev_tree = main_tree.add("👨‍💻 dev")
feature_tree = dev_tree.add(":sparkles: feature/ta-signals-features")
dev_tree.add(":sparkles: feature/news-api-downloader")
dev_tree.add(":sparkles: feature/staking-portfolio-management")
dev_tree.add(":sparkles: bugfix/evaluator-metrics-fix")

console.print(tree)

Branch structure                                                                                    
┗━━ 📦 main (release)                                                                               
    ┗━━ 👨‍💻 dev                                                                                    
        ┣━━ ✨ feature/ta-signals-features                                                          
        ┣━━ ✨ feature/news-api-downloader                                                          
        ┣━━ ✨ feature/staking-portfolio-management                                                 
        ┗━━ ✨ bugfix/evaluator-metrics-fix

## 4. Crediting sources

Some of the components in this library may be based on forum posts, notebooks or ideas made public by the Numerai community. We have done our best to ask all parties who posted a specific piece of code for their permission and credit their work in the documentation. If your code is used in this library without credits, please let us know, so we can add a link to your article/code.

If you are contributing to `numerblox` and are using ideas posted earlier by someone else, make sure to credit them by posting a link to their article/code in documentation.

In [4]:
# hide
# Run this cell to sync all changes with library
from nbdev.export import notebook2script

notebook2script()

Converted 00_misc.ipynb.
Converted 01_download.ipynb.
Converted 02_numerframe.ipynb.
Converted 03_preprocessing.ipynb.
Converted 04_model.ipynb.
Converted 05_postprocessing.ipynb.
Converted 06_modelpipeline.ipynb.
Converted 07_evaluation.ipynb.
Converted 08_key.ipynb.
Converted 09_submission.ipynb.
Converted 10_staking.ipynb.
Converted index.ipynb.
